In [1]:
import pandas as pd
import pickle
import numpy as np
import math
import itertools
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import torch
import random

import sys
from os import path
sys.path.append(path.dirname(path.dirname(path.abspath('../constants'))))
import constants.consts as consts
from model import KPRN, predict
from data.format import format_paths
from data.path_extraction import find_paths_user_to_songs
from recommender import sample_paths

In [2]:
data_path = 'data/' + consts.SONG_IX_DATA_DIR
with open(data_path + 'rs_ix_song_person.dict', 'rb') as handle:
    song_person = pickle.load(handle)
with open(data_path + 'rs_ix_person_song.dict', 'rb') as handle:
    person_song = pickle.load(handle)
with open(data_path + 'rs_ix_song_user.dict', 'rb') as handle:
    song_user = pickle.load(handle)
with open(data_path + 'rs_ix_user_song.dict', 'rb') as handle:
    user_song = pickle.load(handle)
    
mapping_path = 'data/' + consts.SONG_IX_MAPPING_DIR
with open(mapping_path + 'rs_type_to_ix.dict', 'rb') as handle:
    type_to_ix = pickle.load(handle)
with open(mapping_path + 'rs_relation_to_ix.dict', 'rb') as handle:
    relation_to_ix = pickle.load(handle)
with open(mapping_path + 'rs_entity_to_ix.dict', 'rb') as handle:
    entity_to_ix = pickle.load(handle)
with open(mapping_path + 'rs_ix_to_type.dict', 'rb') as handle:
    ix_to_type = pickle.load(handle)
with open(mapping_path + 'rs_ix_to_relation.dict', 'rb') as handle:
    ix_to_relation = pickle.load(handle)
with open(mapping_path + 'rs_ix_to_entity.dict', 'rb') as handle:
    ix_to_entity = pickle.load(handle)


In [3]:
def convert_to_read_format(path):
    '''
    Convert path to readable format
    '''
    new_path = []
    for i,step in enumerate(path):
        if step[1] == consts.SONG_TYPE:
            new_path.append([song_id_to_name[ix_to_entity[step[0]][0]], ix_to_type[step[1]], ix_to_relation[step[2]]])
        else:
            new_path.append([ix_to_entity[step[0]][0], ix_to_type[step[1]], ix_to_relation[step[2]]])
    return new_path

In [4]:
extra_info = pd.read_csv("data/song_dataset/song_extra_info.csv")
song_id_to_name = dict(zip(extra_info.song_id, extra_info.name))
song_id_to_name = {v: k for v,k in song_id_to_name.items() if ( (v,0) in entity_to_ix)}
name_to_song_id = {v: k for k,v in song_id_to_name.items()}

In [5]:
my_songs = ["Edge of Night", "Mausoleum", "Horse to Water", "Thunder", "Angela", "Classy Girls", "In the Morning"]
#Those aren't in our subset, so could maybe force include them in it at some point if we really want to know
my_songs = ["We Are Never Ever Getting Back Together", "Thunder", "Starships", "Counting Stars"]
# print(song_id_to_name['3Yz717mUtR+r4L0zzgDgWnYeFtY9g68ZjKlq1U72KNw='])
# print(name_to_song_id["We Are Never Ever Getting Back Together"])
# print(entity_to_ix[('3Yz717mUtR+r4L0zzgDgWnYeFtY9g68ZjKlq1U72KNw=', 0)])
# print(ix_to_entity[36793])

for my_song in my_songs:
    if my_song in name_to_song_id:
        print(my_song)
        song_id = entity_to_ix[(name_to_song_id[my_song], 0)]
        print(song_id in song_user or song_id in song_person)

We Are Never Ever Getting Back Together
True
Thunder
True
Starships
True
Counting Stars
True


In [6]:
my_songs = ["We Are Never Ever Getting Back Together", "Thunder", "Starships", "Counting Stars", "Fireflies (Owl City (貓頭鷹城市))"]
my_ids = list(map(lambda x: name_to_song_id[x], my_songs))
my_idxs = list(map(lambda x: entity_to_ix[(x,0)], my_ids))
print(my_idxs)

#need to delete entity to add me
# i am user = 63000
my_idx = 63000
user_song[63000] = my_idxs
for song in song_user:
    if my_idx in song_user[song]:
        song_user[song].remove(my_idx)
    

for song_idx in my_idxs:
    if song_idx in song_user and my_idx not in song_user[song_idx]:
        song_user[song_idx].append(my_idx)
    elif song_idx not in song_user:
        song_user[song_idx]=[my_idx]
    print(song_user[song_idx])
    
print(len(entity_to_ix))


[36793, 58881, 55317, 27898, 21337]
[63764, 62426, 64470, 62599, 64472, 64309, 62257, 62258, 63590, 63770, 64477, 62753, 62261, 62605, 63775, 62262, 63428, 62927, 64133, 64642, 64814, 62434, 63952, 64483, 64314, 62268, 63955, 63108, 64661, 63111, 64654, 64821, 64143, 64144, 64320, 64663, 62271, 63787, 63115, 63118, 63610, 62942, 63121, 63443, 64666, 63272, 63125, 62279, 63968, 63616, 64158, 62773, 63972, 64839, 63973, 62775, 62451, 63795, 63978, 62778, 64672, 62289, 62951, 62779, 63278, 63450, 63802, 63803, 64336, 64501, 64338, 63138, 64341, 63139, 62462, 64507, 64858, 62629, 63633, 64175, 64343, 63982, 64680, 62788, 64510, 63989, 63143, 62632, 63640, 63641, 64181, 62466, 63819, 64350, 64868, 62637, 62796, 63147, 64352, 64353, 63148, 63149, 64516, 64354, 64692, 63825, 63998, 64359, 64358, 64361, 64696, 64189, 64190, 63153, 62649, 63154, 64364, 64877, 64197, 62316, 64372, 64199, 62318, 63163, 64526, 62485, 62487, 62322, 62807, 63165, 63314, 64014, 63663, 64205, 64714, 62810, 64017, 6316

In [7]:
#get paths from me to all songs
song_to_paths = find_paths_user_to_songs(my_idx, song_person, person_song, song_user, user_song, 3, 60)
song_to_paths_len5 = find_paths_user_to_songs(my_idx, song_person, person_song, song_user, user_song, 5, 6)
for song in song_to_paths_len5.keys():
    song_to_paths[song].extend(song_to_paths_len5[song])
    
print(len(song_to_paths))

model = KPRN(consts.ENTITY_EMB_DIM, consts.TYPE_EMB_DIM, consts.REL_EMB_DIM, consts.HIDDEN_DIM,
                 len(entity_to_ix), len(type_to_ix), len(relation_to_ix), consts.TAG_SIZE, False)
checkpoint = torch.load("model/" + 'rs_all_sample_5_wp_1.pt', map_location=torch.device("cpu"))
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()
interactions = []

songs_with_paths = list(song_to_paths.keys())
# for song_num in songs_with_paths:
#     for path in song_to_paths[songs_with_paths[song_num]]:
#         print(convert_to_read_format(path))
    

readable_inters = []
for song in songs_with_paths:
    sampled_paths = sample_paths(song_to_paths[song], 5)
    formatted_paths = format_paths(sampled_paths, entity_to_ix, type_to_ix, relation_to_ix)
    interactions.append((formatted_paths, 1))
    
    readable_inter = []
    for path in sampled_paths:
        readable_inter.append(convert_to_read_format(path))
    readable_inters.append(readable_inter)

prediction_scores = predict(model, interactions, 256, "cpu", False, 1)
merged = list(zip(prediction_scores, readable_inters))
s_merged = sorted(merged, key=lambda x: x[0], reverse=True)
for (score, inter) in s_merged:
    print("******")
    print(score)
    if inter[0][5][0] == '#':
        print(inter[0][3][0])
    else:
        print(inter[0][5][0])
    print()
    print(inter)
    print()



1935


<All keys matched successfully>

KPRN(
  (entity_embeddings): Embedding(79877, 64)
  (type_embeddings): Embedding(4, 32)
  (rel_embeddings): Embedding(7, 32)
  (lstm): LSTM(128, 256)
  (linear1): Linear(in_features=256, out_features=256, bias=True)
  (linear2): Linear(in_features=256, out_features=2, bias=True)
)

******
0.9719244241714478
派對動物 (Party Animal)

[[['fLPskRiNL1qHkqlG586osNIfM9EikMu4B9Bs8Bklyic=', 'user', 'user_song'], ['We Are Never Ever Getting Back Together', 'song', 'song_user'], ['LmYdcjmt4ULC4n0hgDsRp/uTRzn68ov+yt/haGLe3kQ=', 'user', 'user_song'], ['派對動物 (Party Animal)', 'song', '#END_RELATION'], ['#', '#PAD_TOKEN', '#PAD_TOKEN'], ['#', '#PAD_TOKEN', '#PAD_TOKEN']], [['fLPskRiNL1qHkqlG586osNIfM9EikMu4B9Bs8Bklyic=', 'user', 'user_song'], ['We Are Never Ever Getting Back Together', 'song', 'song_user'], ['bmc0hxFCxUvp732xh5bj+p/iolZ2GnhDjaa1frb43b0=', 'user', 'user_song'], ['派對動物 (Party Animal)', 'song', '#END_RELATION'], ['#', '#PAD_TOKEN', '#PAD_TOKEN'], ['#', '#PAD_TOKEN', '#PAD_TOKEN']], [['fLPskRiNL1qHkqlG586osNIfM9EikMu4B9Bs8Bklyic=', 'user', 'user_song'], ['We Are Never Ever Getting Back Together', 'song', 'song_user'], ['v9ZGgDqK05OPdRT1uHl7xARTOwsf+oshO07BgGSyUYI=', 'user', 'user_song'], ['以戰止戰', 'song', 'song_user'], ['rtnLWUpQlVT6xu4EUbhmWfHW8/6G+I1dt8XWVpZ3t/A=', 'us

[[['fLPskRiNL1qHkqlG586osNIfM9EikMu4B9Bs8Bklyic=', 'user', 'user_song'], ['Fireflies (Owl City (貓頭鷹城市))', 'song', 'song_user'], ['crUeQ6DYDHHRNYmn+n5Kodfvu9ki0IdHfXHNlZEUPHs=', 'user', 'user_song'], ['Always There', 'song', 'song_user'], ['Wc3sX9RyaEozCuXyy6daSVYgXAseY6HccmvMwRAmkuk=', 'user', 'user_song'], ['直的彎的 (Dimension)', 'song', '#END_RELATION']], [['fLPskRiNL1qHkqlG586osNIfM9EikMu4B9Bs8Bklyic=', 'user', 'user_song'], ['We Are Never Ever Getting Back Together', 'song', 'song_user'], ['bmc0hxFCxUvp732xh5bj+p/iolZ2GnhDjaa1frb43b0=', 'user', 'user_song'], ['直的彎的 (Dimension)', 'song', '#END_RELATION'], ['#', '#PAD_TOKEN', '#PAD_TOKEN'], ['#', '#PAD_TOKEN', '#PAD_TOKEN']], [['fLPskRiNL1qHkqlG586osNIfM9EikMu4B9Bs8Bklyic=', 'user', 'user_song'], ['We Are Never Ever Getting Back Together', 'song', 'song_user'], ['CYb4RkfFA6c2hTfrymGqHECXGPS4cCor0pbWWzIJtaA=', 'user', 'user_song'], ['直的彎的 (Dimension)', 'song', '#END_RELATION'], ['#', '#PAD_TOKEN', '#PAD_TOKEN'], ['#', '#PAD_TOKEN', '#PAD

0.20616960525512695
愛死你

[[['fLPskRiNL1qHkqlG586osNIfM9EikMu4B9Bs8Bklyic=', 'user', 'user_song'], ['We Are Never Ever Getting Back Together', 'song', 'song_user'], ['LmYdcjmt4ULC4n0hgDsRp/uTRzn68ov+yt/haGLe3kQ=', 'user', 'user_song'], ['愛死你', 'song', '#END_RELATION'], ['#', '#PAD_TOKEN', '#PAD_TOKEN'], ['#', '#PAD_TOKEN', '#PAD_TOKEN']], [['fLPskRiNL1qHkqlG586osNIfM9EikMu4B9Bs8Bklyic=', 'user', 'user_song'], ['Thunder', 'song', 'song_person'], ['Jessie J', 'person', 'person_song'], ['Do It Like A Dude', 'song', 'song_user'], ['cm10JDi89WNCj/QNtNEzuMyr3pf8kEVmPBJ46CDi/iI=', 'user', 'user_song'], ['愛死你', 'song', '#END_RELATION']], [['fLPskRiNL1qHkqlG586osNIfM9EikMu4B9Bs8Bklyic=', 'user', 'user_song'], ['We Are Never Ever Getting Back Together', 'song', 'song_user'], ['Jhmx8hXoGJ0doRc3Sm157VHs9fwPd0NRXuv6S7ZA1PM=', 'user', 'user_song'], ['愛死你', 'song', '#END_RELATION'], ['#', '#PAD_TOKEN', '#PAD_TOKEN'], ['#', '#PAD_TOKEN', '#PAD_TOKEN']]]

******
0.20599649846553802
在你眼裡 (In Your Eyes)



艾琳娜 (Elena)

[[['fLPskRiNL1qHkqlG586osNIfM9EikMu4B9Bs8Bklyic=', 'user', 'user_song'], ['We Are Never Ever Getting Back Together', 'song', 'song_user'], ['HmAaZSQ9He3BXNAxkgizK0Z18tHFXPKwLZBVKUitT+c=', 'user', 'user_song'], ['Cold Water (feat. Justin Bieber & MØ)', 'song', 'song_user'], ['WlMRw4Dapq2igAue8Pc8VJaCD0fNdRYmsBZHw3GKcuA=', 'user', 'user_song'], ['艾琳娜 (Elena)', 'song', '#END_RELATION']], [['fLPskRiNL1qHkqlG586osNIfM9EikMu4B9Bs8Bklyic=', 'user', 'user_song'], ['We Are Never Ever Getting Back Together', 'song', 'song_user'], ['Uigz4lVMkS0JiUzt51D3FluM90HIO4RrlqbAMTBXZJQ=', 'user', 'user_song'], ['艾琳娜 (Elena)', 'song', '#END_RELATION'], ['#', '#PAD_TOKEN', '#PAD_TOKEN'], ['#', '#PAD_TOKEN', '#PAD_TOKEN']]]

******
0.11222423613071442
別問我

[[['fLPskRiNL1qHkqlG586osNIfM9EikMu4B9Bs8Bklyic=', 'user', 'user_song'], ['We Are Never Ever Getting Back Together', 'song', 'song_user'], ['frZtrrN1Y2ZqZX3VFiq7FpAvLth3kZJNovT9cmyn2O4=', 'user', 'user_song'], ['別問我', 'song', '#END_RELATION'], 

[[['fLPskRiNL1qHkqlG586osNIfM9EikMu4B9Bs8Bklyic=', 'user', 'user_song'], ['We Are Never Ever Getting Back Together', 'song', 'song_user'], ['HmAaZSQ9He3BXNAxkgizK0Z18tHFXPKwLZBVKUitT+c=', 'user', 'user_song'], ['Cold Water (feat. Justin Bieber & MØ)', 'song', 'song_user'], ['UYsDRcm4UqNQVN+IrTfJX1MDsFCiGu/5EZT6MKb3B4o=', 'user', 'user_song'], ['Believe Me (feat. Bobo & Styles Of Beyond)', 'song', '#END_RELATION']]]

******
0.06142864376306534
銀河

[[['fLPskRiNL1qHkqlG586osNIfM9EikMu4B9Bs8Bklyic=', 'user', 'user_song'], ['We Are Never Ever Getting Back Together', 'song', 'song_user'], ['LmYdcjmt4ULC4n0hgDsRp/uTRzn68ov+yt/haGLe3kQ=', 'user', 'user_song'], ['銀河', 'song', '#END_RELATION'], ['#', '#PAD_TOKEN', '#PAD_TOKEN'], ['#', '#PAD_TOKEN', '#PAD_TOKEN']]]

******
0.0613289438188076
藍色貝殼

[[['fLPskRiNL1qHkqlG586osNIfM9EikMu4B9Bs8Bklyic=', 'user', 'user_song'], ['We Are Never Ever Getting Back Together', 'song', 'song_user'], ['HmAaZSQ9He3BXNAxkgizK0Z18tHFXPKwLZBVKUitT+c=', 'user', 'user_

0.02881673537194729
Brave Shine

[[['fLPskRiNL1qHkqlG586osNIfM9EikMu4B9Bs8Bklyic=', 'user', 'user_song'], ['We Are Never Ever Getting Back Together', 'song', 'song_user'], ['b4ApVHTu/f/mwM1ZGV+LpaOj5GcvMvyDKA1eU1LjUa0=', 'user', 'user_song'], ['Brave Shine', 'song', '#END_RELATION'], ['#', '#PAD_TOKEN', '#PAD_TOKEN'], ['#', '#PAD_TOKEN', '#PAD_TOKEN']]]

******
0.02863229252398014
Headlights (feat. Ilsey)

[[['fLPskRiNL1qHkqlG586osNIfM9EikMu4B9Bs8Bklyic=', 'user', 'user_song'], ['We Are Never Ever Getting Back Together', 'song', 'song_user'], ['2owiLN7VK7szUQvhMAFkgJv5uBtNHKvtOxr8hAZYbCI=', 'user', 'user_song'], ['愛錯', 'song', 'song_user'], ['7Bxnp0ZbIovQD+BR17C3zDBgAA+9HF0fFf7rubcCG8g=', 'user', 'user_song'], ['Headlights (feat. Ilsey)', 'song', '#END_RELATION']]]

******
0.028623096644878387
假裝我們沒愛過

[[['fLPskRiNL1qHkqlG586osNIfM9EikMu4B9Bs8Bklyic=', 'user', 'user_song'], ['We Are Never Ever Getting Back Together', 'song', 'song_user'], ['DVZnLHtUkz2Axh6XpcDhr1dxwBOCMZ57yCyJ7TcbptI='

[[['fLPskRiNL1qHkqlG586osNIfM9EikMu4B9Bs8Bklyic=', 'user', 'user_song'], ['We Are Never Ever Getting Back Together', 'song', 'song_user'], ['v9ZGgDqK05OPdRT1uHl7xARTOwsf+oshO07BgGSyUYI=', 'user', 'user_song'], ['Welcome to TOKYO', 'song', 'song_user'], ['WHW4u19brCw9SV66XI7k2yDp33DwBzCUndYz8AHBP24=', 'user', 'user_song'], ['奉陪', 'song', '#END_RELATION']]]

******
0.004738389980047941
All the Girls

[[['fLPskRiNL1qHkqlG586osNIfM9EikMu4B9Bs8Bklyic=', 'user', 'user_song'], ['We Are Never Ever Getting Back Together', 'song', 'song_user'], ['JXLPxa2007vSS+n75RJZfjJx4LoHEFdc5ahqzzhhyus=', 'user', 'user_song'], ['All the Girls', 'song', '#END_RELATION'], ['#', '#PAD_TOKEN', '#PAD_TOKEN'], ['#', '#PAD_TOKEN', '#PAD_TOKEN']]]

******
0.004615511745214462
Welcome To Mystery

[[['fLPskRiNL1qHkqlG586osNIfM9EikMu4B9Bs8Bklyic=', 'user', 'user_song'], ['We Are Never Ever Getting Back Together', 'song', 'song_user'], ['8arpyGZg/DAjyCJ5IWlQRUHExXe7ykId/WmusVkd3L8=', 'user', 'user_song'], ['Welcome To M